Shauna Smith

Bellevue University-DSC630

Professor: Andrew Hua

Week 10 - "Recommender System"

In [1]:
#Import Libraries
import pandas as pd
import numpy as np

In [2]:
#Load the ratings csv file
dataRatings=pd.read_csv(r"C:\Users\Shaun\OneDrive\Documents\DSC630\ml-latest-small\ratings.csv")

In [3]:
#load the movies file for titles and genres
movieLabels=pd.read_csv(r"C:\Users\Shaun\OneDrive\Documents\DSC630\ml-latest-small\movies.csv")

In [4]:
#Combine files to give titles and genre with ratings
movies=dataRatings.merge(movieLabels, on='movieId', how='left')

In [5]:
#view new df movies to confirm correct labels required
movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [6]:
#creating average rating dataframe
AvgRatings=pd.DataFrame(movies.groupby('title')['rating'].mean())

In [7]:
AvgRatings.head()

,rating
title,
'71 (2014),4.0
'Hellboy': The Seeds of Creation (2004),4.0
'Round Midnight (1986),3.5
'Salem's Lot (2004),5.0
'Til There Was You (1997),4.0


In [8]:
#adjusting for count value dependancy vs. ratings value in the df display
AvgRatings['total ratings']=pd.DataFrame(movies.groupby('title')['rating'].count())

In [9]:
AvgRatings.head()

,rating,total ratings
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


In [10]:
#Create pivot table for the recommender system
movieUser=movies.pivot_table(index='userId',columns='title',values='rating')
movieUser.head(2)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#removing runtime warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#discovering correlation value with selected movie given by user as input
#User stated:("The all time best movie ever is Blade Runner from 1982, but not the director's cut!")
#We entered the movie title as found in dataset
correlate=movieUser.corrwith(movieUser['Blade Runner (1982)'])
correlate.head()

title
'71 (2014)                                NaN
'Hellboy': The Seeds of Creation (2004)   NaN
'Round Midnight (1986)                    NaN
'Salem's Lot (2004)                       NaN
'Til There Was You (1997)                 NaN
dtype: float64

In [29]:
#creating recommendations without NaN values in a new df to assist dataset volume size and pertinance as recommend.
recommend=pd.DataFrame(correlate, columns=['correlation'])
recommend.dropna(inplace=True)
#Now joining total ratings to recommend
recommend=recommend.join(AvgRatings['total ratings'])
recommend.head(10)

,correlation,total ratings
title,,
"'burbs, The (1989)",0.267122,17
(500) Days of Summer (2009),-0.142328,42
*batteries not included (1987),0.718185,7
...And Justice for All (1979),1.000000,3
10 Cent Pistol (2015),1.000000,2
10 Cloverfield Lane (2016),0.406269,14
10 Things I Hate About You (1999),0.039446,54
"10,000 BC (2008)",-0.666063,17
101 Dalmatians (1996),0.273543,47


In [30]:
#filtering the recommend df to only include ratings with more than 100 count in a new df for puposes of reliability as substance
recommendations=recommend[recommend['total ratings']>100].sort_values('correlation', ascending=False).reset_index()

In [31]:
#merging the recommendations of titles with genres to our new df for display purposes
recommendations=recommendations.merge(movies, on='title', how='left')

In [32]:
#dropping duplicates of recommendations by total ratings counts, so as to only show the title with the first instance, 
#and then sorting the df by correlation values for top ten titles for the entry title
recommendations = (recommendations.sort_values('total ratings', key=lambda x: x.eq('movieId'))
        .drop_duplicates('title', keep='first')).sort_values('correlation', ascending=False).reset_index()

In [33]:
#showing top 10 recommended movies by correlation value to the user's entered movie(Blade Runner). 
#This is including the enetered movie at index 0 for reference reasons
recommendations.head(11)

,index,title,correlation,total ratings,userId,movieId,rating,timestamp,genres
0,0,Blade Runner (1982),1.000000,124,16,541,4.5,1377477340,Action|Sci-Fi|Thriller
1,209,Full Metal Jacket (1987),0.539540,102,508,1222,4.5,1268298460,Drama|War
2,228,"Terminator, The (1984)",0.532967,131,15,1240,4.0,1510572646,Action|Sci-Fi|Thriller
3,476,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",0.532582,120,610,4973,4.0,1479543012,Comedy|Romance
4,518,Minority Report (2002),0.513656,120,223,5445,4.0,1226209775,Action|Crime|Mystery|Sci-Fi|Thriller
5,673,Apocalypse Now (1979),0.500307,107,419,1208,3.0,1321854974,Action|Drama|War
6,812,2001: A Space Odyssey (1968),0.473118,109,610,924,4.5,1493850330,Adventure|Drama|Sci-Fi
7,828,Trainspotting (1996),0.464132,102,112,778,3.5,1513989917,Comedy|Crime|Drama
8,924,Terminator 2: Judgment Day (1991),0.463667,224,28,589,4.5,1234515689,Action|Sci-Fi
9,1139,Taxi Driver (1976),0.460977,104,16,111,4.5,1377477446,Crime|Drama|Thriller


# Summary

### Objective 

<u> In your write-up, clearly explain the recommender system process and all steps performed. If you are using a method found online, be sure to reference the source.</u>

In the cell lines above, I give a brief synopsis as comments/cell regarding what each cell is accomplishing at the directive of a recommender system for Movies. In the given example, the user selected "Blade Runner (1982)" from the dataset, an excellent choice! I then entered the movie title, and proceeded to remove all Nan values and duplicate entries from the system recommendations output. This new reccommendations data frame had the yielded recommended movies from the dataset. I then purged the data, to only use recommendations with a "more than 100 user ratings", and cropped for the display of the first 11 sorted results by correlative value. I then showed the top 11 so as to include the original entry for reference, and the proceeding top ten recommendations from the dataset under the prescribed conditions. 

The method I had used was a combination of the provided article "How To Build Your First Recommender System Using Python & MovieLens Dataset" by Amal Nair with Analytics India Magazine(AIM, 2023), and some existing pythonic knowledge to best 'tweak' the code to fit the assignment requirements for top ten movie recommendations as a displayed data frame. 

Sources:
-Nair, Amal (2023), 'How To Build Your First Recommender System Using Python & MovieLens Dataset', ANALYTICS INDIA MAGAZINE PVT LTD & AIM MEDIA HOUSE LLC, https://analyticsindiamag.com/how-to-build-your-first-recommender-system-using-python-movielens-dataset/
